# Importando as bibliotecas

In [ ]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
from db import connection_db as conndb
from db import filters
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm.auto import tqdm
from utils.estracao_interacao import ExtracaoInteracao

tqdm.pandas()

Conectando ao banco de dados do MongoDB

In [ ]:
uri = 'mongodb://localhost:27017/'
db_name = 'dadosVivamente'
col_name = 'dadosSemFiltros'

mongo_connection = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=col_name)
mongo_connection.connect()
collection = mongo_connection.collection

Aplicando pipeline para filtrar os dados

In [ ]:
collection_filters = filters.CollectionFilters(collection)

# Aplicando pipeline 1
collection_filters.apply_pipeline1('dadosComFiltrosIniciais')

# Aplicando pipeline 2 - pega os posts que tenha a data de publicação de no máximo 7 anos e no mínimo 2 anos
collection_filters.apply_pipeline2(7, 2, 'posts7anos2anos')

# Aplicando pipeline 3 - desenrola os posts em documentos individuais e cria novas colunas
collection_filters.apply_pipeline3('postsComBDIAndInfos')

# Aplicando pipeline 4 - aplica o filtro para selecionar apenas os posts que tem uma data de publicação válida
collection_filters.apply_pipeline4('postsComBDIAndInfosFiltroDataPosts')

# Seleciona a data final e inicial para filtrar os posts que foram publicados em um intervalo de 6 meses antes da coleta
data_inicio = datetime(2017, 12, 1)
data_fim = data_inicio - relativedelta(months=11)

# Aplicando pipeline 5 - aplica o filtro para selecionar apenas os posts que foram publicados em um intervalo de 6 meses antes da coleta
collection_filters.apply_pipeline5('postsFiltradosPorData', data_inicio, data_fim)

collection = collection_filters.collection

Aplica pipeline para ver a distribuição dos dados

In [ ]:
# Função para obter quantidade de usuários por nível e gênero
def obter_quantidades_nivel_e_genero(nivel, genero=None):
    if genero:
        return collection_filters.count_users_by_gender('nivel', '$eq', nivel, genero)
    return collection_filters.quant_users_cat('nivel', '$eq', nivel)


niveis = [0, 1, 2, 3]

# Dicionários para armazenar as quantidades por nível e gênero
quantidades = {nivel: obter_quantidades_nivel_e_genero(nivel) for nivel in niveis}
quantidades_masculino = {nivel: obter_quantidades_nivel_e_genero(nivel, 'M') for nivel in niveis}
quantidades_feminino = {nivel: obter_quantidades_nivel_e_genero(nivel, 'F') for nivel in niveis}

for nivel in niveis:
    print(f'Quantidade de Usuários com nível {nivel}:', quantidades[nivel])
    print(f'Quantidade de Usuários do sexo Masculino com nível {nivel}:', quantidades_masculino[nivel])
    print(f'Quantidade de Usuários do sexo Feminino com nível {nivel}:', quantidades_feminino[nivel], '\n')

Criando o gráficos com a quantidade de usuários por nível de depressão

In [ ]:
# Plotar gráfico de barras com a quantidade de usuários por nível
def plotar_grafico_quantidades_por_nivel(quantidades):
    fig, ax = plt.subplots(figsize=(8, 6))
    niveis_labels = [f'Nível {nivel}' for nivel in niveis]
    quantidades_list = [quantidades[nivel] for nivel in niveis]

    ax.bar(niveis_labels, quantidades_list, color=['#4CAF50', '#FFC107', '#2196F3', '#F44336'])

    ax.set_ylabel('Quantidade de Usuários')
    ax.set_xlabel('Nível')
    ax.set_title('Quantidade de Usuários por score BDI-II')
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Adicionar rótulos nas barras
    for i, v in enumerate(quantidades_list):
        ax.text(i, v + max(quantidades_list) * 0.01, str(v), ha='center', va='bottom')

    plt.savefig('dados/filtro_nivel/quantidade_usuarios_por_nivel_depressao_6meses.png', bbox_inches='tight')
    plt.show()


# Plotar gráfico de barras com a quantidade de usuários por nível e gênero
def plotar_grafico_quantidades_por_nivel_e_genero(quantidades_masculino, quantidades_feminino):
    bar_width = 0.35
    niveis_labels = [f'Nível {nivel}' for nivel in niveis]
    r1 = np.arange(len(niveis))
    r2 = [x + bar_width for x in r1]

    fig, ax = plt.subplots(figsize=(10, 6))

    ax.bar(r1, [quantidades_masculino[nivel] for nivel in niveis], color='#1f77b4', width=bar_width, edgecolor='grey',
           label='Masculino')
    ax.bar(r2, [quantidades_feminino[nivel] for nivel in niveis], color='#ff7f0e', width=bar_width, edgecolor='grey',
           label='Feminino')

    ax.set_xlabel('Nível', fontweight='bold')
    ax.set_ylabel('Quantidade de Usuários', fontweight='bold')
    ax.set_title('Quantidade de Usuários por score BDI-II e Gênero')

    ax.set_xticks([r + bar_width / 2 for r in range(len(niveis))])
    ax.set_xticklabels(niveis_labels)

    ax.legend()

    # Adicionar rótulos nas barras
    for i in range(len(niveis)):
        ax.text(r1[i] - bar_width / 2, quantidades_masculino[niveis[i]] + 1, quantidades_masculino[niveis[i]],
                ha='center', va='bottom', fontsize=9)
        ax.text(r2[i] - bar_width / 2, quantidades_feminino[niveis[i]] + 1, quantidades_feminino[niveis[i]],
                ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.savefig('dados/filtro_nivel/quantidade_usuarios_por_nivel_depressao_e_genero_6meses.png', dpi=300)
    plt.show()


# Chamadas para plotar os gráficos
plotar_grafico_quantidades_por_nivel(quantidades)
plotar_grafico_quantidades_por_nivel_e_genero(quantidades_masculino, quantidades_feminino)

Criando um DataFrame com os dados dos usuários

In [ ]:
documentos = collection.find()
df = pd.DataFrame(list(documentos))

In [ ]:
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

# Preencher os NaNs com 0
df[colunas_para_converter] = df[colunas_para_converter].fillna(0)

df[colunas_para_converter] = df[colunas_para_converter].astype('int64')

In [ ]:
# Filtrar pela coluna suicida maior ou igual a 2
df = df[df['suicida'] >= 1]

In [ ]:
# Filtrar pela coluna mes maior ou igual a 11
df = df[df['mes'] >= 10]

In [ ]:
# Dropar usuario com quantidade de amigos nula
df = df.dropna(subset=['quantAmigos'])

In [ ]:
# Adicionando colunas de interações
extrator = ExtracaoInteracao(df)

# Extrair as interações e obter o DataFrame atualizado
df = extrator.extract_interactions()

In [ ]:
df.head()

In [ ]:
# Obter uma visão geral das colunas e seus tipos
print(df.info())

In [ ]:
# Estatísticas básicas das variáveis numéricas
print(df.describe())

In [ ]:
print(df.isnull().sum())

In [ ]:
# Converter a coluna sexo para numérico
df['sexo'] = df['sexo'].map({'F': 0, 'M': 1})

df['diaDaSemanaNumerico'] = df['postCreated_time'].dt.weekday
df.head()

In [ ]:
from utils.text_treatment import TextTreatment

tratamento_texto = TextTreatment()
df['postMessageLimpo'] = df['postMessage'].fillna('').progress_apply(
    lambda texto: tratamento_texto.preprocessamento_texto(texto) if texto else '')

In [ ]:
from utils.busca_palavras import BuscaPalavras

busca_palavras = BuscaPalavras()

# Faz a busca exata
resultado = busca_palavras.string_matching(df['postMessageLimpo'], 'dados/datasets/termos_depressivos_pt_br.txt')

# Adicionar a coluna quantPalavrasDepressivas ao DataFrame, contando quantas palavras depressivas foram encontradas
df['quantPalavrasDepressivas'] = resultado.apply(lambda x: len(x.split(', ')) if x else 0)

In [ ]:
df_agg = df.groupby(['id_usuario']).agg(
    idade=('idade', 'first'),
    sexo=('sexo', 'first'),
    pessimismo=('pessimismo', 'first'),
    tristeza=('tristeza', 'first'),
    fracasso=('fracasso', 'first'),
    prazer=('prazer', 'first'),
    culpa=('culpa', 'first'),
    punicao=('punicao', 'first'),
    estima=('estima', 'first'),
    critica=('critica', 'first'),
    suicida=('suicida', 'first'),
    choro=('choro', 'first'),
    agitacao=('agitacao', 'first'),
    interesse=('interesse', 'first'),
    indecisao=('indecisao', 'first'),
    desvalorizacao=('desvalorizacao', 'first'),
    energia=('energia', 'first'),
    sono=('sono', 'first'),
    irritabilidade=('irritabilidade', 'first'),
    apetite=('apetite', 'first'),
    concentracao=('concentracao', 'first'),
    fadiga=('fadiga', 'first'),
    int_sexo=('int_sexo', 'first'),
    quantAmigos=('quantAmigos', 'first'),
    quantPosts=('id_usuario', 'count'),
    mediaStory=('postStory', lambda x: round(x.notnull().mean(), 2)),
    mediaMsg=('postMessage', lambda x: round(x.notnull().mean(), 2)),
    horaMedia=('hora', lambda x: round(x.mean(), 2)),
    modaDiaSemana=('diaDaSemanaNumerico', lambda x: x.mode()[0]),
    quantPalavrasDepre=('quantPalavrasDepressivas', lambda x: round(x.mean(), 2))
).reset_index()

In [ ]:
df_agg.head()

In [ ]:
df_agg.drop(columns=['id_usuario'], inplace=True)

In [ ]:
# Definir um limite mínimo para quantPosts (por exemplo, excluir usuários com menos de 5 posts)
limite_inferior = 20
df_agg = df_agg[df_agg['quantPosts'] >= limite_inferior]

In [ ]:
df_agg.to_csv('df_agg.csv', index=False)

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df_agg

# Escalonamento com StandardScaler (média 0 e desvio padrão 1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Exibindo os resultados escalonados
print("StandardScaler:\n", X_scaled)

In [ ]:
from sklearn.decomposition import PCA

# Aplicar PCA com base na variância acumulada desejada (ex: 95%)
pca = PCA(n_components=0.90)
X_pca = pca.fit_transform(X_scaled)
# Quantidade de componentes principais escolhidos automaticamente
print("Número de componentes principais:", X_pca.shape[1])

In [ ]:
from sklearn.cluster import KMeans

k_range = range(1, 12)

# Aplicando o modelo K-Means para cada valor de K
k_means_var = [KMeans(n_clusters=k).fit(X_pca) for k in k_range]

In [ ]:
# Ajustando o centróide do cluster para cada modelo K-Means
centroids = [model.cluster_centers_ for model in k_means_var]

In [ ]:
from scipy.spatial.distance import cdist

# Calculando a distância euclidiana de cada ponto de dado para o centróide
k_euclid = [cdist(X_pca, cent, 'euclidean') for cent in centroids]

# Para cada ponto, pegamos a menor distância de qualquer centróide
dist = [np.min(ke, axis=1) for ke in k_euclid]

In [ ]:
inertias = [model.inertia_ for model in k_means_var]

# Curva de Elbow
plt.figure()
plt.plot(k_range, inertias, 'bx-')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo para Encontrar o Número Ideal de Clusters')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

# Testando K=2
kmeans_2 = KMeans(n_clusters=2, random_state=42)
kmeans_2.fit(X_pca)
labels_2 = kmeans_2.labels_
silhouette_avg_2 = silhouette_score(X_pca, labels_2)
print(f"Silhouette Score para K=2: {silhouette_avg_2}")

# Testando K=3
kmeans_3 = KMeans(n_clusters=3, random_state=42)
kmeans_3.fit(X_pca)
labels_3 = kmeans_3.labels_
silhouette_avg_3 = silhouette_score(X_pca, labels_3)
print(f"Silhouette Score para K=3: {silhouette_avg_3}")

# Testando K=4
kmeans_4 = KMeans(n_clusters=4, random_state=42)
kmeans_4.fit(X_pca)
labels_4 = kmeans_4.labels_
silhouette_avg_4 = silhouette_score(X_pca, labels_4)
print(f"Silhouette Score para K=4: {silhouette_avg_4}")

# Testando K=5
kmeans_5 = KMeans(n_clusters=5, random_state=42)
kmeans_5.fit(X_pca)
labels_5 = kmeans_5.labels_
silhouette_avg_5 = silhouette_score(X_pca, labels_5)
print(f"Silhouette Score para K=5: {silhouette_avg_5}")

# Testando K=6
kmeans_6 = KMeans(n_clusters=6, random_state=42)
kmeans_6.fit(X_pca)
labels_6 = kmeans_6.labels_
silhouette_avg_6 = silhouette_score(X_pca, labels_6)
print(f"Silhouette Score para K=6: {silhouette_avg_6}")

# Testando K=7
kmeans_7 = KMeans(n_clusters=7, random_state=42)
kmeans_7.fit(X_pca)
labels_7 = kmeans_7.labels_
silhouette_avg_7 = silhouette_score(X_pca, labels_7)
print(f"Silhouette Score para K=7: {silhouette_avg_7}")

# Testando K=8
kmeans_8 = KMeans(n_clusters=8, random_state=42)
kmeans_8.fit(X_pca)
labels_8 = kmeans_8.labels_
silhouette_avg_8 = silhouette_score(X_pca, labels_8)
print(f"Silhouette Score para K=8: {silhouette_avg_8}")

In [ ]:
# Aplicando K-Means nos dados transformados pelo PCA
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X_pca)

# Acessando os centróides
centroids = kmeans.cluster_centers_

# Verificando as dimensões dos centróides
print("Dimensões de X_pca:", X_pca.shape)
print("Dimensões dos centróides:", centroids.shape)

In [ ]:
# Atribuir os rótulos (clusters) ao dataframe original com K=4
df_agg['cluster'] = kmeans.labels_

# Verificar os dados com o cluster atribuído
df_agg.head()

In [ ]:
df_agg.to_csv('df_agg_cluster.csv', index=False)

In [ ]:
# Agrupar os dados por cluster e calcular as médias das variáveis por cluster
cluster_summary = df_agg.groupby('cluster').mean()

# Exibir o resumo
print(cluster_summary)

In [ ]:
silhouette_values = silhouette_score(X_pca, kmeans.labels_, metric='euclidean')
print("Silhouette Score geral:", silhouette_values)

# Verificar a silhueta de cada ponto
from sklearn.metrics import silhouette_samples

silhouette_vals = silhouette_samples(X_pca, kmeans.labels_)

# Agrupar os valores de silhueta por cluster
df_agg['silhouette_values'] = silhouette_vals
silhouette_summary = df_agg.groupby('cluster')['silhouette_values'].mean()
print(silhouette_summary)

In [ ]:
import seaborn as sns

plt.figure(figsize=(20, 15), dpi=300)
sns.heatmap(df_agg.corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.show()
plt.close

In [ ]:
# Definir o limiar de correlação forte (exemplo: 0.7)
limiar_correlacao = 0.5

# Calcular a matriz de correlação
correlacao = df_agg.corr()

# Pegar os pares de correlação que têm correlação maior ou igual ao limiar
correlacao_forte = correlacao[(correlacao >= limiar_correlacao) | (correlacao <= -limiar_correlacao)]

# Remover os NaNs que não são fortes o suficiente
correlacao_forte = correlacao_forte.dropna(how='all', axis=0).dropna(how='all', axis=1)

# Exibir a matriz de correlações fortes
plt.figure(figsize=(15, 10), dpi=300)
sns.heatmap(correlacao_forte, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação - Correlações Fortes')
plt.show()
plt.close()

# Se quiser pegar os pares de features com correlações fortes para análise posterior
pares_fortes = np.where((correlacao >= limiar_correlacao) | (correlacao <= -limiar_correlacao))
pares_fortes = [(correlacao.index[i], correlacao.columns[j], correlacao.iloc[i, j])
                for i, j in zip(*pares_fortes) if i != j]

# Exibir os pares de correlações fortes
for feature1, feature2, corr_value in pares_fortes:
    print(f'{feature1} e {feature2}: correlação = {corr_value:.2f}')

In [ ]:
# Calcular a matriz de correlação de Spearman
correlacao_spearman = df_agg.corr(method='spearman')

# Plotar a matriz de correlação de Spearman
plt.figure(figsize=(20, 15), dpi=300)
sns.heatmap(correlacao_spearman, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Matriz de Correlação - Spearman (Relações Não Lineares)')
plt.show()
plt.close()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Escolher algumas variáveis de interesse para visualizar as relações
variaveis_interesse = ['idade', 'sexo', 'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao',
                       'estima', 'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao', 'desvalorizacao',
                       'energia', 'sono', 'irritabilidade', 'apetite', 'concentracao', 'fadiga', 'int_sexo',
                       'quantAmigos', 'quantPosts', 'mediaStory', 'mediaMsg', 'horaMedia', 'modaDiaSemana',
                       'quantPalavrasDepre']

# Gerar gráficos de dispersão para ver relações não lineares
for var1 in variaveis_interesse:
    for var2 in variaveis_interesse:
        if var1 != var2:
            plt.figure(figsize=(8, 6))
            sns.scatterplot(x=df_agg[var1], y=df_agg[var2])
            plt.title(f'Relação entre {var1} e {var2}')
            plt.show()
            plt.close()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Supondo que você esteja tentando prever um cluster (criado com K-Means)
X = df_agg.drop(columns=['cluster'])
y = df_agg['cluster']

# Separar dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinar uma árvore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

# Fazer previsões
y_pred = tree.predict(X_test)

# Avaliar o modelo
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Criar um modelo SVM com kernel RBF (não linear)
svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train, y_train)

# Fazer previsões
y_pred_svm = svm_model.predict(X_test)

# Avaliar o modelo
print(classification_report(y_test, y_pred_svm))